In [13]:
import ast
from math import floor, ceil
from itertools import product

In [14]:
class SnailFishNumber:

    def __init__(self, left, right, reduce=False):
        self.left, self.right = left, right
        self.left_depth = 0 if type(self.left) == int else self.left.depth
        self.right_depth = 0 if type(self.right) == int else self.right.depth
        if reduce:
            self._reduce()

    @classmethod
    def from_ls(cls, ls, reduce=False):
        left, right = cls._parse(ls)
        return cls(left, right, reduce)

    @classmethod
    def from_str(cls, string, reduce=False):
        return cls.from_ls(ast.literal_eval(string), reduce)

    @property
    def magnitude(self):
        left = self.left if type(self.left) == int else self.left.magnitude
        right = self.right if type(self.right) == int else self.right.magnitude
        return 3 * left + 2 * right

    @property
    def depth(self):
        return 1 + max(self.left_depth, self.right_depth)

    @property
    def should_split(self):
        left = self.left > 9 if self.left_depth == 0 else self.left.should_split
        right = self.right > 9 if self.right_depth == 0 else self.right.should_split
        return left or right

    def _reduce(self):
        flag = True
        while flag:
            flag = False
            if self.depth == 5:
                flag = True
                self._explode()
            elif self.should_split:
                self._split()
                flag = True

    def _split(self):
        if self.left > 9 if self.left_depth == 0 else self.left.should_split:
            if self.left_depth == 0:
                self.left = SnailFishNumber(floor(self.left / 2), ceil(self.left / 2))
                self.left_depth = 1
            else:
                self.left._split()
                self.left_depth = self.left.depth
        elif self.right > 9 if self.right_depth == 0 else self.right.should_split:
            if self.right_depth == 0:
                self.right = SnailFishNumber(floor(self.right / 2), ceil(self.right / 2))
                self.right_depth = 1
            else:
                self.right._split()
                self.right_depth = self.right.depth

    def _explode(self):
        if self.depth == 2:
            if self.left_depth == 1:
                if self.right_depth == 1:
                    self.right.left += self.left.right
                else:
                    self.right += self.left.right
                out = ('left', self.left.left)
                self.left = 0
                self.left_depth = 0
            elif self.right_depth == 1:
                if self.left_depth == 1:
                    self.left.right += self.right.left
                else:
                    self.left += self.right.left
                out = ('right', self.right.right)
                self.right = 0
                self.right_depth = 0
            return out
        if self.left_depth >= self.right_depth:
            side, val = self.left._explode()
            self.left_depth = self.left.depth
            if side == 'left':
                return side, val
            elif side == 'right':
                if self.right_depth == 0:
                    self.right += val
                else:
                    self.right.left_addition(val)
                return '', 0
            else:
                return '', 0
        else:
            side, val = self.right._explode()
            self.right_depth = self.right.depth
            if side == 'right':
                return side, val
            elif side == 'left':
                if self.left_depth == 0:
                    self.left += val
                else:
                    self.left.right_addition(val)
                return '', 0
            else:
                return '', 0

    def left_addition(self, val):
        if self.left_depth == 0:
            self.left += val
        else:
            self.left.left_addition(val)

    def right_addition(self, val):
        if self.right_depth == 0:
            self.right += val
        else:
            self.right.right_addition(val)


    @classmethod
    def _parse(cls, ls):
        if len(ls) != 2:
            raise ValueError(f'len of input is not 2, {ls}')
        left, right = ls
        if type(left) != int:
            left = cls.from_ls(left)
        if type(right) != int:
            right = cls.from_ls(right)
        return left, right


    def __add__(self, other):
        return SnailFishNumber(self, other, reduce=True)

    def __str__(self):
        return f's[{self.left}, {self.right}]'

    def __repr__(self):
        return str(self)

In [15]:
with open('../inputs/input18.txt', 'r') as f:
    strings = f.readlines()

In [16]:
snail_numbers = [SnailFishNumber.from_str(string) for string in strings]
snail_numbers

[s[s[0, 6], s[s[s[4, 0], s[6, 6]], s[s[2, 2], 9]]],
 s[s[9, s[s[1, 6], s[6, 0]]], s[s[1, s[0, 8]], s[s[0, 8], s[9, 8]]]],
 s[s[s[0, s[2, 1]], 3], s[s[s[2, 4], s[1, 2]], s[7, 5]]],
 s[s[s[s[8, 3], s[8, 5]], s[s[7, 8], s[5, 5]]], s[9, 2]],
 s[s[8, s[1, 9]], s[s[s[9, 9], s[9, 2]], 1]],
 s[s[s[s[3, 7], s[2, 1]], s[0, 9]], 4],
 s[s[s[s[3, 8], s[6, 0]], s[0, 7]], s[s[s[6, 3], s[2, 0]], 9]],
 s[s[s[9, s[7, 0]], s[8, s[9, 6]]], s[s[5, 6], 4]],
 s[s[s[s[3, 6], s[3, 6]], s[0, 2]], s[s[s[8, 3], 9], s[s[3, 4], 8]]],
 s[s[7, s[8, 4]], 1],
 s[6, s[s[3, s[5, 6]], s[0, 6]]],
 s[s[s[7, s[4, 7]], s[s[4, 5], s[4, 3]]], s[s[5, 5], s[0, s[4, 2]]]],
 s[s[s[0, s[2, 9]], s[s[2, 4], s[4, 8]]], s[s[8, s[9, 5]], s[s[9, 6], 0]]],
 s[s[s[s[2, 0], s[9, 7]], s[s[3, 2], 0]], s[7, 7]],
 s[s[5, s[2, 1]], s[s[3, s[5, 1]], s[s[8, 5], s[1, 8]]]],
 s[s[s[s[9, 7], 6], s[s[7, 8], 7]], s[s[s[6, 8], 9], s[s[9, 5], 7]]],
 s[s[4, 2], s[s[s[0, 1], s[7, 2]], s[s[0, 2], s[5, 5]]]],
 s[s[1, 8], s[s[5, s[7, 9]], s[s[3, 1], s[7, 1]]]]

In [17]:
snail_numbers = [SnailFishNumber.from_str(string) for string in strings]
out = snail_numbers[0]
for add_number in snail_numbers[1:]:
    out = out + add_number
out.magnitude #antwoord van 1

4088

In [20]:
max_val = 0
for str1, str2 in product(strings, repeat=2):
    n1 = SnailFishNumber.from_str(str1)
    n2 = SnailFishNumber.from_str(str2)
    new_number = n1 + n2
    val = (n1+n2).magnitude
    if val > max_val:
        max_val = val
max_val

4536